In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
!pip install linkpreview

In [ ]:
%%writefile app.py
#!/usr/bin/env python
# coding: utf-8

#!/usr/bin/env python
# coding: utf-8


import numpy as np
import pandas as pd
import streamlit as st
from linkpreview import link_preview
from PIL import Image
import urllib.request

st.write("""
# Wild Cinema
""")

@st.cache(show_spinner=False)
def imports():
    movies=pd.read_csv('/content/drive/MyDrive/movies2.csv')
    distances=pd.read_csv('/content/drive/MyDrive/distances.csv')
    genome_score = pd.read_csv('/content/drive/MyDrive/genome-scores.csv') #
    MB=movies[movies.tags==True].reset_index(drop=True)
    links=pd.read_csv("/content/drive/MyDrive/imageLLL.csv")
    return (movies,distances,genome_score,MB,links)

movies,distances,genome_score,MB,links=imports()

def userinput():
    out=st.text_input("Entrez un nom de film")
    return out.lower()

entry1=userinput()

@st.cache(show_spinner=False)
def pos(entry):
    possibilities=movies[movies.title2.str.contains(entry)].reset_index(drop=True)
    return possibilities

possibilities=pos(entry1)
if len(possibilities)==0:
    st.write("Désolé, pas de film avec ce nom.")    
else:      
    if len(possibilities)==1:
        target=possibilities.title[0]    
    else:
        choice=st.selectbox(label="Cherchez vous un de ces films?",options=possibilities.title)

        if st.button('Valider'):
            target=choice
    try:
        target_index=movies.loc[movies.title==target].index[0]  
        if movies.loc[target_index,"tags"]:
            st.write("Recommandations par mot-clés")
            IdFilmChosen=movies.loc[target_index,"movieId"]
            top_20_tags_id = genome_score[genome_score['movieId'] == IdFilmChosen].sort_values('relevance', ascending = False).iloc[0:20,1]

            most_relevant_movies = genome_score[genome_score['tagId'].isin(list(top_20_tags_id))].groupby(
                'movieId', as_index = False).mean().drop('tagId', axis = 1)
            df2=MB.loc[most_relevant_movies.sort_values('relevance', ascending = False).head(6).index.values,["title","movieId"]]    
            st.write(df2.title)
            

        st.write("Recommandations par genres")
        df1=movies.loc[np.array(distances.loc[target_index]).reshape(10,),["title","movieId"]].head(6)
        st.write(df1.title)

        df = pd.concat([df1,df2],axis=0).drop_duplicates()
        image=links.loc[links.movieId.isin(df.movieId),"final_link"]
        liste=[]
        for i in image:
            pic=Image.open(urllib.request.urlopen(link_preview(i).absolute_image))
            liste.append(pic)
        
        st.image(liste,width=120)    
    except:
        pass

Writing app.py


In [ ]:
!ls

app.py	drive  sample_data


In [ ]:
!ngrok authtoken 1lBXr6r9fgcKUjRdHmXJgoMvAsv_74MDGC8cxudQ48YPqcrYa

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
!streamlit run --server.port 80 app.py &>/dev/null&

In [ ]:
!pgrep streamlit

334


In [ ]:
# Setup a tunnel to the streamlit port 8501
publ_url = ngrok.connect(port='8501')

In [ ]:
publ_url

<NgrokTunnel: "http://97a2357d7c68.ngrok.io" -> "http://localhost:80">